<a href="https://colab.research.google.com/github/gskumlehn/timelens/blob/main/GNews_Fetch_All.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GNews → Excel (todas as páginas em uma aba)

**Input único:** `query`.

**Token:** o notebook lê o token da API do GNews a partir da variável de ambiente `GNEWS_TOKEN`.

**Saída:** um arquivo `.xlsx` com todas as notícias de todas as páginas em uma única planilha e *download automático* no Colab.

**Como usar no Colab:**
1. Em *Runtime → Restart and run all* (ou execute célula a célula).
2. Antes de executar a célula final, defina `GNEWS_TOKEN` no ambiente do Colab (ex.: `import os; os.environ['GNEWS_TOKEN']='SEU_TOKEN_AQUI'`).
3. Informe a `query` no formulário e execute.

In [4]:
# @title Instalações (se necessário)
!pip -q install requests pandas openpyxl

In [5]:
import os
import re
import requests
import pandas as pd
from datetime import datetime
from google.colab import files

GNEWS_TOKEN = "1d4fbd4762fc0b33ddf1b0ba67fd7292"
BASE_URL = "https://gnews.io/api/v4/search"

def _sanitize_filename(name: str) -> str:
    name = re.sub(r"\s+", "_", name.strip())
    name = re.sub(r"[^A-Za-z0-9._-]", "", name)
    return name[:150] or "resultado"

def search_gnews(query, lang="pt", country="br", max_results=100, from_date=None, to_date=None, sort_by="publishedAt", page=1):
    API_KEY = GNEWS_TOKEN
    if not API_KEY:
        raise RuntimeError("Variável de ambiente GNEWS_TOKEN não definida.")

    params = {
        "q": query,
        "token": API_KEY,
        "lang": lang,
        "in": "title,description,content",
        "country": country,
        "max": max_results,
        "sortby": sort_by,
        "page": page,
        "expand": "content",
    }

    if from_date:
        params["from"] = from_date
    if to_date:
        params["to"] = to_date

    resp = requests.get(BASE_URL, params=params, timeout=60)
    resp.raise_for_status()
    return resp.json()

def fetch_and_save_all(query, lang="pt", country="br", max_per_page=100, from_date=None, to_date=None, sort_by="publishedAt"):
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    base = _sanitize_filename(query)
    if from_date or to_date:
        output_file = f"{base}-from{from_date}-to{to_date}-{ts}.xlsx"
    else:
        output_file = f"{base}-{ts}.xlsx"

    dfs = []
    page = 1
    while True:
        data = search_gnews(query, lang, country, max_per_page, from_date, to_date, sort_by, page)
        articles = data.get("articles", [])
        if not articles:
            break
        dfs.append(pd.json_normalize(articles))
        page += 1

    if not dfs:
        raise RuntimeError("Nenhuma notícia encontrada para a query informada.")

    all_df = pd.concat(dfs, ignore_index=True)
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        all_df.to_excel(writer, sheet_name="results", index=False)

    return output_file

print("Pronto. Use a célula abaixo para rodar sua query.")

Pronto. Use a célula abaixo para rodar sua query.


In [6]:
# @title Rodar busca e baixar Excel
query = "Banco do Brasil" # @param {type:"string"}
lang = "pt"        # @param ["pt"]
country = "br"     # @param ["br"]
sort_by = "publishedAt"  # @param ["publishedAt", "relevance"]
max_per_page = 100  # @param {type:"integer"}
from_date = "\"2025-08-04T12:00:00.000Z\""    # @param {type:"string"}
to_date = "\"2025-08-06T12:00:00.000Z\""      # @param {type:"string"}

xlsx_path = fetch_and_save_all(
    query=query,
    lang=lang,
    country=country,
    max_per_page=max_per_page,
    from_date=from_date,
    to_date=to_date,
    sort_by=sort_by,
)
print("Arquivo gerado:", xlsx_path)
files.download(xlsx_path)

Arquivo gerado: Banco_do_Brasil-from"2025-08-04T12:00:00.000Z"-to"2025-08-06T12:00:00.000Z"-20250812-171920.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>